<a href="https://colab.research.google.com/github/BirenderPanwar/tsai_eva4p2/blob/master/5-HumanPoseEstimation/notebooks/S5_HPE_Pytorch_To_Onnx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple Baseline for Human Pose Estimation and Tracking

1. Pytorch Resnet50 model trained in MPII dataset is used.
2. Model is converted into ONNX format
3. ONNX model is quantized to reduce the model file size from 130MB to around 65MB

## Reference Materials:
1. https://github.com/Microsoft/human-pose-estimation.pytorch
2. Paper: https://arxiv.org/pdf/1804.06208.pdf
3. Model is downloaded from: https://onedrive.live.com/?authkey=%21AFkTgCsr3CT9%2D%5FA&id=56B9F9C97F261712%2110709&cid=56B9F9C97F261712



In [1]:
# mount gdrive
mount_drive = True
if mount_drive:
    from google.colab import drive
    drive.mount('/content/gdrive') 

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import os
os.chdir('/content/gdrive/My Drive/TSAI/EVA4_Phase2/session5/notebooks/')
print(os.getcwd())

/content/gdrive/My Drive/TSAI/EVA4_Phase2/session5/notebooks


In [3]:
# Import standard packages
import torch
import torchvision

In [4]:
# Basic file to import for HPE
import _init_paths # set the system path for './human-pose-estimation.pytorch/lib'
from core.config import config
from core.config import update_config
import models

In [5]:
# To autoreload all te custom files when modified
import autoreload
%load_ext autoreload
%autoreload

In [6]:
# Necessary files path
HPE_ROOT = './human-pose-estimation.pytorch'
CONFIG_FILE = f'./{HPE_ROOT}/experiments/mpii/resnet50/256x256_d256x3_adam_lr1e-3.yaml'
MODEL_BASEPATH = f'./hpe_trained_model'

In [7]:
# update the configutaion for HPE
update_config(CONFIG_FILE)

In [8]:
# loading HPE pytorch model
model = eval('models.'+config.MODEL.NAME+'.get_pose_net')(config, is_train=False)
model.load_state_dict(torch.load(f'{MODEL_BASEPATH}/pose_resnet_50_256x256.pth.tar', map_location=torch.device('cpu')))

<All keys matched successfully>

In [9]:
# Fxn to check model size
get_model_size = lambda filename: os.path.getsize(filename)/1e6

In [10]:
model_path = f'{MODEL_BASEPATH}/pose_resnet_50_256x256_pytorch.pt'
torch.save(model, model_path)
print(f'Pytorch model size (MB): {get_model_size(model_path):0.2f}')

Pytorch model size (MB): 136.34


# Exporting pytorch model into ONNX format

In [11]:
!pip install onnx onnxruntime

In [12]:
import torch.onnx

batch_size = 1
x = torch.randn(batch_size, 3, 256, 256, requires_grad=True)
output = model(x)

onnx_model_path = f'{MODEL_BASEPATH}/pose_resnet_50_256x256_onxx.onnx'

# Export the model
torch.onnx.export(model,                     # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  onnx_model_path,   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable lenght axes
                                'output' : {0 : 'batch_size'}})

In [13]:
print(f'ONXX model size (MB): {get_model_size(onnx_model_path):0.2f}')

ONXX model size (MB): 136.25


# Quantizing onnx model to further reducing the model size for AWS Lambda fitness

In [14]:
import onnx

# Import the ONNX model
onnx_model = onnx.load(onnx_model_path)
onnx.checker.check_model(onnx_model)

In [15]:
from onnxruntime.quantization import quantize
from onnxruntime.quantization import QuantizationMode

onnx_quantized_model_path = f'{MODEL_BASEPATH}/pose_resnet_50_256x256_quantized.onnx'

onnx_quantized_model = quantize(onnx_model, quantization_mode=QuantizationMode.IntegerOps, static=False)
onnx.save(onnx_quantized_model, onnx_quantized_model_path)

In [16]:
print(f'ONXX Quantized model size (MB): {get_model_size(onnx_quantized_model_path):0.2f}')

ONXX Quantized model size (MB): 65.93


### Refer S5_HPE_OnnxRuntime_Inferencing.ipynb notebook for inferencing the model using OnnxRuntime